Pima Indians Diabetes Database 문제

In [1]:
import pandas as pd
g = pd.read_csv('diabetes.csv')
g.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
g.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


로지스틱 회귀는 0 또는 1로 구분 , 타깃 0,1로 되어있는지 확인이 필요함

In [3]:
g['Outcome'].head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

결측치, 이상치가 있는지 확인

In [4]:
# 결측치 확인 방법
g.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [5]:
print(g.columns)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')


In [10]:
# 로지스틱 회귀 구현을 위한 모듈 불러오기
import statsmodels.api as sm
# 모델 구축 및 학습
model_glm = sm.Logit.from_formula('Outcome ~ Pregnancies + Glucose + BloodPressure + SkinThickness + Insulin + BMI + DiabetesPedigreeFunction + Age', data=g).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.470993
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      759
Method:                           MLE   Df Model:                            8
Date:                Mon, 01 Jul 2024   Pseudo R-squ.:                  0.2718
Time:                        16:53:07   Log-Likelihood:                -361.72
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 9.652e-54
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -8.4047      0.717    -11.728      0.000      -9.809      -7.000
Pregnancies                  0.1232      0.032      3.840      0.000       0.060       0.186
Glucose                      0.0352      0.004      9.481      0.000       0.028       0.042
BloodPressure               -0.0133      0.005     -2.540      0.011      -0.024      -0.003
SkinThickness                0.0006      0.007      0.090      0.929      -0.013       0.014
Insulin                     -0.0012      0.001     -1.322      0.186      -0.003       0.001
BMI                          0.0897      0.015      5.945      0.000       0.060       0.119
DiabetesPedigreeFunction     0.9452      0.299      3.160      0.002       0.359       1.531
Age                          0.0149      0.009      1.593      0.111      -0.003       0.033
============================================================================================
"""

1. p-value (P>|z|) 값이 SkinThickness, Insulin, Age가 0.05보다 큼 -> 의미가 있는 값인가?

=> SkinThickness, Insulin, Age 제거하고 다시 해보면 어떨까?

2. 이상치를 가지고 있던 값이 있었는데 제거하면 좋아지지않을까?

4개의 변수를 제외 후 로지스틱 회귀를 사용

In [11]:
# 로지스틱 회귀 구현을 위한 모듈 불러오기
import statsmodels.api as sm
# 모델 구축 및 학습
model_glm = sm.Logit.from_formula('Outcome ~ Pregnancies + Glucose + DiabetesPedigreeFunction', data=g).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.502132
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Outcome   No. Observations:                  768
Model:                          Logit   Df Residuals:                      764
Method:                           MLE   Df Model:                            3
Date:                Mon, 01 Jul 2024   Pseudo R-squ.:                  0.2237
Time:                        16:55:10   Log-Likelihood:                -385.64
converged:                       True   LL-Null:                       -496.74
Covariance Type:            nonrobust   LLR p-value:                 6.685e-48
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -6.1748      0.469    -13.152      0.000      -7.095      -5.255
Pregnancies                  0.1302      0.026      5.001      0.000       0.079       0.181
Glucose                      0.0364      0.003     10.987      0.000       0.030       0.043
DiabetesPedigreeFunction     1.0013      0.276      3.625      0.000       0.460       1.543
============================================================================================
"""

In [9]:
# predict() 메소드를 이용해 예측 값(확률) 생성
y_prob = model_glm.predict(g.iloc[:, 0:2])

y_prob.head()

PatsyError: predict requires that you use a DataFrame when predicting from a model
that was created using the formula api.

The original error message returned by patsy is:
Error evaluating factor: NameError: name 'DiabetesPedigreeFunction' is not defined
    Outcome ~ Pregnancies + Glucose + DiabetesPedigreeFunction
                                      ^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
# 예측 값(확률)을 반올림을 통해 1과 0으로 표현한 뒤 list 타입으로 변경
y_pred = list(map(round, y_prob))
# 예측 값(클래스) 확인
y_pred[0:5]